# Identification Task 

## What we need

We need a function called check_msm_identification that it easy to use and performs the identification check described in [this paper](https://arxiv.org/pdf/1907.13093.pdf). The different variants (e.g. different methods of sampling uniformly from likelihood level sets) can be selected via the optional arguments of check_ml_identification.  The output will either be a dictionary (if it is a small set of outputs that every user will want) or a results object similar to the result of estimate_ml (if there are many different test statistics).

## Task 1: Planning

- Write down which model specific inputs a user has to supply in order to do an identification check. The names should be aligned with estimate_ml where possible. It will definitely be a likelihood function and a result of estimate_msm but there might be more. 
- Write down which kinds of outputs a user will get, what they mean and how they should be visualized in a paper (plots, tables, ...). 
- Write docstrings for check_ml_identification before you actually implement it
- Adjust our [simple example](https://estimagic.readthedocs.io/en/stable/getting_started/estimation/first_msm_estimation_with_estimagic.html) such that it has a second variable that can be arbitrarily correlated with x (i.e. add an identification problem)
- Start to write a tutorial in a notebook that shows how the new function will be used and what the outputs mean

## Remarks

- You can for now assume that the model parameters (params) are a 1d numpy array. We talk about making this more flexible later. 
- The idea behind writing the documentation first is that it lets you focus completely on a user friendly interface and a high level understanding. Also, we will probably ask for changes after you show us your proposed interface. If you had already implemented it, you would have to change it.

In [2]:
from scipy.stats import ncx2
from scipy.linalg import sqrtm
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import scipy.stats.qmc as qmc

In [3]:
# task 1 - interface - inputs and outputs
def check_msm_identification():
    """ Do detecting of identification failure in moment condition models. 
    Performs the identification check as described in Forneron, J. J. (2019).
    Introduces the quasi-Jacobian matrix computed as a slope of a linear 
    approximation of the moments of the estimate of the identified set. It is 
    asymptotically singular when local or global identification fails and equivalent 
    to the usual Jacobian matrix which has full rank when model is point and locally 
    identified.

    Args
        simulate_moments (callable) – Function that takes as inputs model parameters, 
            data and potentially other keyword arguments and returns a pytree with 
            simulated moments.  If the function returns a dict containing the key 
            "simulated_moments" we only use the value corresponding to that key. Other 
            entries are stored in the log database if you use logging.
        simulate_moments_kwargs (dict) – Additional keyword arguments for simulate_moments 
            with, for example, data on dependent and independent variables from the model 
            specification.
        params (pytree) – A pytree containing the estimated parameters of the model.
            Pytrees can be a numpy array, a pandas Series, a DataFrame with “value” column, a 
            float and any kind of (nested) dictionary or list containing these elements.
        weights (str) – One of “diagonal” (default), “identity” or “optimal”.  Note that 
            “optimal” refers to the asymptotically optimal weighting matrix and is often 
            not a good choice due to large finite sample bias.
        bandwidth (float) - By default is calculated in the form of sqrt(2log(log[n])/n). 
            Required for the calculation of quasi-jacobian matrix.
        kernel (callable) - By default  is the uniform kernel K(U) which is indicator 
            function for |U|<=1. Required for the calculation of quasi-jacobian matrix.
        cutoff (float) - By default is calculated in the form sqrt(2log[n]/n). Required 
            for identification category selection.
        draws (float)  - The number of draws for sampling on level sets. Supposed to be 
            sufficiently large.
        sampling (str) - Methods of sampling uniformly from likelihood level sets. One of 
            the available options for direct approach using "sobol" or "halton" sequence 
            or adaptive sampling by "population_mc".
        significance (float) - The significance level with default level 5%.
        H0 (dict) - Required for subvector inference. For example, b10 = 0.
        reparametrization - .
        logging (pathlib.Path, str or False) – Path to sqlite3 file (which typically has 
            the file extension .db. If the file does not exist, it will be created. The 
            dashboard can only be used when logging is used.
        log_options (dict) – Additional keyword arguments to configure the logging.

    Returns
        dict: The estimated quasi-Jacobian, singular values, identification category, subvestor 
            inference test output and confidence set.
"""


    # 1. quasi-Jacobian Matrix
    # 1.1 set the integration grid and evaluate the moments on thr grid, select draws on the level set
    # 1.2 compute the intercept and slope
    # 1.3 compute the variance


    # 2. Identification Category Selection
    # 2.1 compute singular values
    # 2.2 number of values grater than cutoff



    # 3. Subvector Inference
    # 3.1 test statistic
    # 3.2 hypothesis, confidence set

estimate_msm
https://estimagic.readthedocs.io/en/stable/reference_guides/index.html

In [5]:
# CALCULATE NECESSARY INPUTS (as in identification_check_with_estimagic.ipynb)
import pandas as pd
import numpy as np
import estimagic as em

rng = np.random.default_rng(seed=0)

def simulate_data(params, n_draws, rng,correlation=0.7):

    mu = np.array([0.0, 0.0])
    var_cov = np.array([
            [  1, correlation],
            [ correlation,  1],
        ])
    x = rng.multivariate_normal(mu, var_cov, size=n_draws)
    x1 = x[:,0]
    x2 = x[:,1]
    e = rng.normal(0, params.loc["sd", "value"], size=n_draws)
    y = params.loc["intercept", "value"] + params.loc["slope1", "value"] * x1 + params.loc["slope2", "value"] + e
    return pd.DataFrame({"y": y, "x1": x1, "x2": x2})

true_params = pd.DataFrame(
    data=[[2, -np.inf], [-1, -np.inf], [-1, -np.inf], [1, 1e-10]],
    columns=["value", "lower_bound"],
    index=["intercept", "slope1", "slope2", "sd"],
)

true_params = pd.DataFrame(
    data=[[2, -np.inf], [-1, -np.inf], [-1, -np.inf], [1, 1e-10]],
    columns=["value", "lower_bound"],
    index=["intercept", "slope1", "slope2", "sd"],
)

data = simulate_data(true_params, n_draws=100, rng=rng)

def calculate_moments(sample):
    moments = {
        "y_mean": sample["y"].mean(),
        "x1_mean": sample["x1"].mean(),
        "x2_mean": sample["x2"].mean(),
        "yx1_mean": (sample["y"] * sample["x1"]).mean(),
        "yx2_mean": (sample["y"] * sample["x2"]).mean(),
        "y_sqrd_mean": (sample["y"] ** 2).mean(),
        "x1_sqrd_mean": (sample["x1"] ** 2).mean(),
        "x2_sqrd_mean": (sample["x1"] ** 2).mean(),
    }
    return pd.Series(moments)

empirical_moments = calculate_moments(data)


def simulate_moments(params, n_draws=10_000, seed=0, data=False):
    rng = np.random.default_rng(seed)
    sim_data = simulate_data(params, n_draws, rng)
    sim_moments = calculate_moments(sim_data)
    
    if data==False:
        return sim_moments
    else:
        return sim_moments, sim_data

moments_cov = em.get_moments_cov(
    data, calculate_moments, bootstrap_kwargs={"n_draws": 5_000, "seed": 0}
)

start_params = true_params.assign(value=[100, 100, 100, 100])

res = em.estimate_msm(
    simulate_moments,
    empirical_moments,
    moments_cov,
    start_params,
    optimize_options={"algorithm":"scipy_lbfgsb"},
)

In [8]:
from estimagic.estimation.msm_weighting import get_weighting_matrix

msm_params = res.params['value'] # estimated coefficient vector
n_params = len(msm_params) # number of estimated parameters
n = data.shape[0] # number of observations
B = 10000 # number of draws


sampling = "sobol" # for step 2.i)
bandwidth = 2 * math.log(math.log(n)) / n # for step 2.i)
weights = 'diagonal' # for weighting matrix in step 2.i)
kernel = 'uniform' # for step 2.ii) linear approximation

# 2.i Draws on the level set
## Sampling for step 2.i)
#### 1. Direct approach
 - draw values from the space - either randomly or pseudo-randomly (Sobol or Halton)
 - assign weights proportionally to the bandwidth criterion (indicator function)
 - drawback - effective sample size can be samll relative to the parameter space; especially when the dimention of \theta is moderately large

#### 2. Adaptive Sampling by Population Monte Carlo
- constructing a sequence of proposal distributions with higher acceptance rate
- to do later

In [134]:
if sampling not in ['random','sobol','halton','population_mc']:
    raise NotImplementedError("Custom sampling is not yet implemented.")

# direct approach
if sampling in ['random','sobol','halton']:
    if sampling=="random":
        random_sequences = np.random.random((B,n_params))
        params_draws = [list(msm_params)]*B + 2 * (random_sequences - 1 / 2)

    elif sampling=="sobol":
        sobol_sequences = qmc.Sobol(d = n_params, scramble = True) # d - dimention - number of estimated parameters
        sobol_sequences = sobol_sequences.random(n=B) # instead of n=B try power of 2 (for Sobol sequence)
        params_draws = [list(msm_params)]*B + 2 * (sobol_sequences - 1 / 2)

    else:
        raise NotImplementedError("Sampling with Halton sequences is not yet implemented.")

    
    # weighting the resulted draws
    objs = [np.nan] * B  # Store GMM objective values
    moms = [[np.nan] * n_params]  * B # Store sample moments mom
    Vs = [np.full([n_params, n_params], np.nan) for i in range(B)] # Store variances V

    for b in range(B): # Evaluate the moments on the grid
        mm, sim_data = simulate_moments(pd.DataFrame(data = params_draws[b], index = list(msm_params.index), columns = ["value"]), n_draws=10_000, seed=0, data=True)
        # ???????? cannot use the em.get_moments_cov because the data generated for moments (in simulate_moments) was not saved
        V = em.get_moments_cov(sim_data, calculate_moments, bootstrap_kwargs={"n_draws": 5_000, "seed": 0})
        moms[b] = mm
        Vs[b] = V
        W, internal_weights = get_weighting_matrix( # weighting matrix
            moments_cov=V,
            method=weights,
            empirical_moments=mm,
            return_type="pytree_and_array",
        )
        
        objs[b] = np.dot(mm.T, np.linalg.solve(W, mm))
        
    # Select draws on the level set
    ind = np.array(objs) <= bandwidth # why it was objs - min(objs) in the code? according to the paper it should be objs <= bandwidth 
    ind = [i for i, x in enumerate(ind) if x]
    grid_sub = params_draws[ind]
    moms_sub = [moms[i] for i in ind]
    Vs_sub = [Vs[i] for i in ind] 


# adaptive sampling
elif sampling=="population_mc":
    raise NotImplementedError("Adaptive Sampling by Population Monte Carlo is not yet implemented.")

else:
    raise NotImplementedError("Custom sequences are not yet implemented.")

c:\Users\aida4\Anaconda3\envs\assignment_5\lib\site-packages\scipy\stats\_qmc.py:1078: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  warnings.warn("The balance properties of Sobol' points require"


TypeError: '<=' not supported between instances of 'list' and 'float'

# 2.ii) Linear approximation
## Kernels for step 2.ii)
 - uniform
 - Epanchnikov- to do later
 - cosine - to do later

In [147]:
grid_sub

array([[-0.34796216, -0.96126163, -0.2366117 ,  1.3666092 ],
       [-0.0836938 ,  0.14177115,  0.74147786,  0.90494826],
       [-0.13637438, -0.39994483,  0.35566129,  1.63163665],
       [ 1.14155937,  0.02938015,  0.27657808,  1.15313479]])

In [151]:
moms_sub

[y_mean         -0.574559
 x1_mean        -0.006568
 x2_mean        -0.003578
 yx1_mean       -0.924803
 yx2_mean       -0.647135
 y_sqrd_mean     3.072987
 x1_sqrd_mean    0.981403
 x2_sqrd_mean    0.981403
 dtype: float64,
 y_mean          0.659304
 x1_mean        -0.006568
 x2_mean        -0.003578
 yx1_mean        0.144577
 yx2_mean        0.106572
 y_sqrd_mean     1.274723
 x1_sqrd_mean    0.981403
 x2_sqrd_mean    0.981403
 dtype: float64,
 y_mean          0.226333
 x1_mean        -0.006568
 x2_mean        -0.003578
 yx1_mean       -0.376346
 yx2_mean       -0.258246
 y_sqrd_mean     2.851829
 x1_sqrd_mean    0.981403
 x2_sqrd_mean    0.981403
 dtype: float64,
 y_mean          1.421067
 x1_mean        -0.006568
 x2_mean        -0.003578
 yx1_mean        0.031959
 yx2_mean        0.028996
 y_sqrd_mean     3.348481
 x1_sqrd_mean    0.981403
 x2_sqrd_mean    0.981403
 dtype: float64]

In [148]:
# where is the kernel part???
# check on how this Chebyshev (minimax) approxiation problem is solved
X = np.column_stack([[1] * len(grid_sub), grid_sub])
coef = np.linalg.solve(np.dot(X.T, X), np.dot(X.T, moms_sub)) # A_n and B_n #!!!!! not sure whether this is always how the solution is calculated
# Here the Kernel K is uniform , re-weigthing is not needed
Bn = coef[1: ].T

In [150]:
X

array([[ 1.        , -0.34796216, -0.96126163, -0.2366117 ,  1.3666092 ],
       [ 1.        , -0.0836938 ,  0.14177115,  0.74147786,  0.90494826],
       [ 1.        , -0.13637438, -0.39994483,  0.35566129,  1.63163665],
       [ 1.        ,  1.14155937,  0.02938015,  0.27657808,  1.15313479]])